In [3]:
# Code courtesy: Lucas Calestini

In [81]:
import requests
import pandas as pd
import numpy as np
import json
from tqdm import tqdm_notebook as tqdm
import time
from pandas import json_normalize

In [2]:
# playlists_df = pd.read_csv('https://media.githubusercontent.com/media/halmasieh/Spotify_Analysis/main/Resources/playlist_frequency.csv')
# tracks_df = pd.read_csv('https://github.com/halmasieh/Spotify_Analysis/blob/main/Resources/cleaned_tracks.csv?raw=true')

In [48]:
# playlists_df.to_csv('playlists_df.csv', index=False)
# tracks_df.to_csv('tracks_df.csv', index=False)

In [54]:
unmatched = playlists_df[playlists_df['track_uri'].isin(tracks_df['id'])==False]

In [56]:
len(playlists_df) - len(unmatched)

14694

In [58]:
tracks_to_query = unmatched['track_uri'].tolist()

In [111]:
def get_song_info(track_ids):
    headers = {
        'Authorization': 'Bearer <insert token>',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    params = {'ids': track_ids}
    return requests.get('https://api.spotify.com/v1/tracks', headers=headers, params=params)

def get_song_features(track_ids):
    headers = {
        'Authorization': 'Bearer <insert token>',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    params = {'ids': track_ids}
    return requests.get('https://api.spotify.com/v1/audio-features', headers=headers, params=params)

def _parse_track(rjson):
    return rjson['tracks']

def _parse_features(rjson):
    return rjson['audio_features']

In [65]:
## tests

r1 = get_song_info('0XUfyU2QviPAs6bxSpXYG4')
r2 = get_song_features('0XUfyU2QviPAs6bxSpXYG4')

In [100]:
main_tracks = []
main_features = []

In [126]:
# loop through every 50 tracks
for i in tqdm(range(len(main_tracks),len(tracks_to_query)+50,50)):
    
    time.sleep(0.5)
    
    tracks = tracks_to_query[i-50:i]
    
    # transform list to comma-separated str
    track_ids = ','.join(tracks)
    
    # query info for those tracks
    track_request = get_song_info(track_ids)
    if track_request.status_code != 200:
        print (track_request.status_code, track_request.text, track_request.headers)
        break
    
    # append to main list of tracks
    main_tracks.extend(_parse_track(track_request.json()))

    # query features for those tracks
    feature_request = get_song_features(track_ids)
    if feature_request.status_code != 200:
        print (feature_request.status_code, feature_request.text, track_request.headers)
        break
        
    # append to main list of features for tracks
    main_features.extend(_parse_features(feature_request.json()))

<ipython-input-126-88f07281daaa>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(main_tracks),len(tracks_to_query)+50,50)):


In [137]:
not_null_main_tracks = []
track_id_errors = []

In [138]:
for t in main_tracks:
    try:
        if t.keys():
            not_null_main_tracks.append(t)
    except AttributeError:
        track_id_errors.append(t)
        pass

In [145]:
not_null_main_feat = []
track_id_feat_errors = []

for t in main_features:
    try:
        if t.keys():
            not_null_main_feat.append(t)
    except AttributeError:
        track_id_feat_errors.append(t)
        pass

In [146]:
len(main_features), len(not_null_main_feat)

(51449, 51447)

In [140]:
len(main_tracks), len(not_null_main_tracks)

(51599, 51595)

In [143]:
main_tracks_df = json_normalize(not_null_main_tracks)

In [147]:
main_features_df = json_normalize(not_null_main_feat)

In [152]:
main_tracks_df.head()

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,preview_url,track_number,type,uri,album.album_type,album.artists,album.available_markets,album.external_urls.spotify,album.href,album.id,album.images,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify
0,[{'external_urls': {'spotify': 'https://open.s...,[],1,207626,True,https://api.spotify.com/v1/tracks/5XJJdNPkwmbU...,5XJJdNPkwmbUwE79gv0NxK,False,Gold Digger,1,None,4,track,spotify:track:5XJJdNPkwmbUwE79gv0NxK,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/4GRDFQ9HRoO0by8...,https://api.spotify.com/v1/albums/4GRDFQ9HRoO0...,4GRDFQ9HRoO0by8H0r2a3I,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Late Registration,2005-01-01,day,21,album,spotify:album:4GRDFQ9HRoO0by8H0r2a3I,USUM70500143,https://open.spotify.com/track/5XJJdNPkwmbUwE7...
1,[{'external_urls': {'spotify': 'https://open.s...,[],1,173986,False,https://api.spotify.com/v1/tracks/1xznGGDReH1o...,1xznGGDReH1oQq0xzbwXa3,False,One Dance,2,None,12,track,spotify:track:1xznGGDReH1oQq0xzbwXa3,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/3hARKC8cinq3mZL...,https://api.spotify.com/v1/albums/3hARKC8cinq3...,3hARKC8cinq3mZLLAEaBh9,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Views,2016-05-06,day,20,album,spotify:album:3hARKC8cinq3mZLLAEaBh9,USCM51600028,https://open.spotify.com/track/1xznGGDReH1oQq0...
2,[{'external_urls': {'spotify': 'https://open.s...,[],1,250373,False,https://api.spotify.com/v1/tracks/0XUfyU2QviPA...,0XUfyU2QviPAs6bxSpXYG4,False,Yeah!,0,None,2,track,spotify:track:0XUfyU2QviPAs6bxSpXYG4,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/0vO0b1AvY49CPQy...,https://api.spotify.com/v1/albums/0vO0b1AvY49C...,0vO0b1AvY49CPQyVisJLj0,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Confessions,2004-03-08,day,17,album,spotify:album:0vO0b1AvY49CPQyVisJLj0,USAR10301423,https://open.spotify.com/track/0XUfyU2QviPAs6b...
3,[{'external_urls': {'spotify': 'https://open.s...,[JP],1,225800,False,https://api.spotify.com/v1/tracks/6O6M7pJLABmf...,6O6M7pJLABmfBRoGZMu76Y,False,Roses (feat. ROZES),31,https://p.scdn.co/mp3-preview/939d9371fbd0b706...,1,track,spotify:track:6O6M7pJLABmfBRoGZMu76Y,album,[{'external_urls': {'spotify': 'https://open.s...,[JP],https://open.spotify.com/album/3ShQFl9FladFKlo...,https://api.spotify.com/v1/albums/3ShQFl9FladF...,3ShQFl9FladFKlonwPGZFc,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",The Chainsmokers- Japan Special Edition,2016-08-10,day,7,album,spotify:album:3ShQFl9FladFKlonwPGZFc,USQX91500801,https://open.spotify.com/track/6O6M7pJLABmfBRo...
4,[{'external_urls': {'spotify': 'https://open.s...,[],1,354320,False,https://api.spotify.com/v1/tracks/1AhDOtG9vPSO...,1AhDOtG9vPSOmsWgNW0BEY,False,Bohemian Rhapsody - Remastered 2011,20,None,11,track,spotify:track:1AhDOtG9vPSOmsWgNW0BEY,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/1TSZDcvlPtAnekT...,https://api.spotify.com/v1/albums/1TSZDcvlPtAn...,1TSZDcvlPtAnekTaItI3qO,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",A Night At The Opera (2011 Remaster),1975-11-21,day,12,album,spotify:album:1TSZDcvlPtAnekTaItI3qO,GBUM71029604,https://open.spotify.com/track/1AhDOtG9vPSOmsW...


In [149]:
main_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.646,0.696,1,-5.572,0,0.3420,0.01920,0.00000,0.0762,0.625,92.880,audio_features,5XJJdNPkwmbUwE79gv0NxK,spotify:track:5XJJdNPkwmbUwE79gv0NxK,https://api.spotify.com/v1/tracks/5XJJdNPkwmbU...,https://api.spotify.com/v1/audio-analysis/5XJJ...,207401,4
1,0.791,0.619,1,-5.886,1,0.0532,0.00784,0.00423,0.3510,0.371,103.989,audio_features,1xznGGDReH1oQq0xzbwXa3,spotify:track:1xznGGDReH1oQq0xzbwXa3,https://api.spotify.com/v1/tracks/1xznGGDReH1o...,https://api.spotify.com/v1/audio-analysis/1xzn...,173987,4
2,0.881,0.788,2,-4.669,1,0.1680,0.02120,0.00000,0.0377,0.592,104.997,audio_features,0XUfyU2QviPAs6bxSpXYG4,spotify:track:0XUfyU2QviPAs6bxSpXYG4,https://api.spotify.com/v1/tracks/0XUfyU2QviPA...,https://api.spotify.com/v1/audio-analysis/0XUf...,250373,4
3,0.716,0.806,4,-7.602,1,0.0466,0.03610,0.00413,0.2440,0.347,100.033,audio_features,6O6M7pJLABmfBRoGZMu76Y,spotify:track:6O6M7pJLABmfBRoGZMu76Y,https://api.spotify.com/v1/tracks/6O6M7pJLABmf...,https://api.spotify.com/v1/audio-analysis/6O6M...,225800,4
4,0.414,0.404,0,-9.928,0,0.0499,0.27100,0.00000,0.3000,0.224,71.105,audio_features,1AhDOtG9vPSOmsWgNW0BEY,spotify:track:1AhDOtG9vPSOmsWgNW0BEY,https://api.spotify.com/v1/tracks/1AhDOtG9vPSO...,https://api.spotify.com/v1/audio-analysis/1AhD...,354320,4


In [154]:
main_tracks_df.to_csv('main_tracks_df.csv.gz', index=False, compression='gzip')
main_features_df.to_csv('main_features_df.csv.gz', index=False, compression='gzip')

In [151]:
pd.set_option('max_columns', 100)

In [155]:
all_joined = main_tracks_df.merge(main_features_df, on='id', how='left')

In [156]:
print (len(all_joined), len(main_tracks_df), len(main_features_df))

51695 51595 51447


In [160]:
all_joined[all_joined['id']=='09TcIuH1ZO7i4vicWKoaN2']

,artists,available_markets,disc_number,duration_ms_x,explicit,href,id,is_local,name,popularity,preview_url,track_number,type_x,uri_x,album.album_type,album.artists,album.available_markets,album.external_urls.spotify,album.href,album.id,album.images,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type_y,uri_y,track_href,analysis_url,duration_ms_y,time_signature
3584,[{'external_urls': {'spotify': 'https://open.s...,[],1,232146,False,https://api.spotify.com/v1/tracks/09TcIuH1ZO7i...,09TcIuH1ZO7i4vicWKoaN2,False,Papaoutai,0,None,2,track,spotify:track:09TcIuH1ZO7i4vicWKoaN2,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/4WW3ME0M2dUoAK5...,https://api.spotify.com/v1/albums/4WW3ME0M2dUo...,4WW3ME0M2dUoAK5hmbXsN7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",racine carrée,2013-01-01,day,13,album,spotify:album:4WW3ME0M2dUoAK5hmbXsN7,BET671300160,https://open.spotify.com/track/09TcIuH1ZO7i4vi...,0.733,0.818,10.0,-7.222,0.0,0.0859,0.0241,0.0,0.0636,0.253,116.019,audio_features,spotify:track:09TcIuH1ZO7i4vicWKoaN2,https://api.spotify.com/v1/tracks/09TcIuH1ZO7i...,https://api.spotify.com/v1/audio-analysis/09Tc...,232147.0,4.0
3585,[{'external_urls': {'spotify': 'https://open.s...,[],1,232146,False,https://api.spotify.com/v1/tracks/09TcIuH1ZO7i...,09TcIuH1ZO7i4vicWKoaN2,False,Papaoutai,0,None,2,track,spotify:track:09TcIuH1ZO7i4vicWKoaN2,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/4WW3ME0M2dUoAK5...,https://api.spotify.com/v1/albums/4WW3ME0M2dUo...,4WW3ME0M2dUoAK5hmbXsN7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",racine carrée,2013-01-01,day,13,album,spotify:album:4WW3ME0M2dUoAK5hmbXsN7,BET671300160,https://open.spotify.com/track/09TcIuH1ZO7i4vi...,0.733,0.818,10.0,-7.222,0.0,0.0859,0.0241,0.0,0.0636,0.253,116.019,audio_features,spotify:track:09TcIuH1ZO7i4vicWKoaN2,https://api.spotify.com/v1/tracks/09TcIuH1ZO7i...,https://api.spotify.com/v1/audio-analysis/09Tc...,232147.0,4.0
3684,[{'external_urls': {'spotify': 'https://open.s...,[],1,232146,False,https://api.spotify.com/v1/tracks/09TcIuH1ZO7i...,09TcIuH1ZO7i4vicWKoaN2,False,Papaoutai,0,None,2,track,spotify:track:09TcIuH1ZO7i4vicWKoaN2,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/4WW3ME0M2dUoAK5...,https://api.spotify.com/v1/albums/4WW3ME0M2dUo...,4WW3ME0M2dUoAK5hmbXsN7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",racine carrée,2013-01-01,day,13,album,spotify:album:4WW3ME0M2dUoAK5hmbXsN7,BET671300160,https://open.spotify.com/track/09TcIuH1ZO7i4vi...,0.733,0.818,10.0,-7.222,0.0,0.0859,0.0241,0.0,0.0636,0.253,116.019,audio_features,spotify:track:09TcIuH1ZO7i4vicWKoaN2,https://api.spotify.com/v1/tracks/09TcIuH1ZO7i...,https://api.spotify.com/v1/audio-analysis/09Tc...,232147.0,4.0
3685,[{'external_urls': {'spotify': 'https://open.s...,[],1,232146,False,https://api.spotify.com/v1/tracks/09TcIuH1ZO7i...,09TcIuH1ZO7i4vicWKoaN2,False,Papaoutai,0,None,2,track,spotify:track:09TcIuH1ZO7i4vicWKoaN2,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/4WW3ME0M2dUoAK5...,https://api.spotify.com/v1/albums/4WW3ME0M2dUo...,4WW3ME0M2dUoAK5hmbXsN7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",racine carrée,2013-01-01,day,13,album,spotify:album:4WW3ME0M2dUoAK5hmbXsN7,BET671300160,https://open.spotify.com/track/09TcIuH1ZO7i4vi...,0.733,0.818,10.0,-7.222,0.0,0.0859,0.0241,0.0,0.0636,0.253,116.019,audio_features,spotify:track:09TcIuH1ZO7i4vicWKoaN2,https://api.spotify.com/v1/tracks/09TcIuH1ZO7i...,https://api.spotify.com/v1/audio-analysis/09Tc...,232147.0,4.0


In [163]:
deduped = all_joined.drop_duplicates(subset=['uri_x','id'])

In [164]:
print (len(deduped), len(all_joined), all_joined['id'].nunique(), deduped['id'].nunique())

51545 51695 51545 51545


In [167]:
deduped.to_csv('new_tracks.csv.gz', index=False, compression='gzip')

In [8]:
## playlist: 66K --> get all tracks information API

## tracks: 586K

## matching 15K

## 51K unmatched 
# (1) there is some noise in the id
# (2) you don't have those tracks in the tracks (get that info from API)

In [ ]:
## concatenate (just make sure the columns have the same name and are aligned)
pd.concat([cleaned_df, new_tracks], axis=0, sort=False)